In [ ]:
!pip install -r "requirements_unix.txt"

Processing ./dadk_light_3.10.tar.bz2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

$$\min \sum_{i,j} x_i \cdot w_{ij} \cdot x_j$$


$$R \leq \sum_{i} x_i \cdot r_i$$


$$\sum_{i} x_i = \frac{N}{2}$$


In [ ]:
# Cargar los datos del dataset
df = pd.read_csv('dataset_precios_acciones_DEMO.csv')

# Eliminar la primera fila porque no es un valor numérico
prices = df.filter(like='Cierre')

# Calcular los retornos diarios y la matriz de covarianza
# La función pct_change() calcula el cambio porcentual entre filas consecutivas, días en este caso
returns = prices.pct_change().dropna()
cov_matrix = returns.cov().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(prices.columns)
asset_names = prices.columns.tolist()

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty):
    var_shape_set = VarShapeSet(BitArrayShape('x', (n_assets,)))
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()
    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            if i != j:
                w_ij = cov_matrix[i, j]
                H_cuad.add_term(w_ij, ('x', i), ('x', j))

    # Construir H_half para representar la restricción
    H_half = BinPol()
    for i in range(n_assets):
        H_half.add_term(1, ('x', i))
    H_half.add_term(-n_assets / 2, ())
    H_half.power(2)
    H_half.multiply_by_scalar(factor_penalty)

    # Combinar los términos para formar el QUBO
    QUBO = BinPol()
    QUBO.add(H_cuad)
    QUBO.add(H_half)

    return QUBO


# Coeficiente de penalización para la restricción H_half
factor_penalty = 50
QUBO = build_qubo(cov_matrix, n_assets, factor_penalty)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(number_iterations=200000, number_runs=10, scaling_bit_precision=32, auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING)
solution_list = solver.minimize(QUBO)
print(solution_list.solver_times)

# Función para preparar y presentar el resultado
def prep_result(solution_list, QUBO, silent=False):
    solution = solution_list.min_solution
    penalty_value = QUBO.compute(solution.configuration)
    if not silent:
        print(f'QUBO value: {penalty_value}')
    return penalty_value, solution

# Función para reportar la solución
def report(penalty_value, solution, asset_names, silent=False):
    if not silent:
        if penalty_value < 0.01:
            print('Chosen portfolio:')
            selected_assets = [asset_names[i] for i, bit in enumerate(solution.configuration) if bit > 0.5]
            print(selected_assets)
        else:
            print("Invalid solution, please solve again!")

# Utilizar las funciones de resultado y reporte
penalty_value, solution = prep_result(solution_list, QUBO)
report(penalty_value, solution, asset_names)

TypeError: 'int' object is not iterable